# Benoit's Problem with various RTO Algorithms

In [83]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import approx_fprime

# 1. Model Adaptation

## A. Benoit Model (Modified Version) for Real Time Optimization without constraint

In [84]:
# Plant Model 
def Benoit_Model(u,theta):
    f = theta[0] * u[0] ** 2 + theta[1] * u[1] ** 2
    return f


def con1_model(u,theta):
    g1 = 1. - theta[2]*u[0] + theta[3]*u[1] ** 2
    return -g1

# Actual Plant System
def Benoit_System_1(u):
    f = u[0] ** 2 + u[1] ** 2 + u[0] * u[1] + np.random.normal(0., np.sqrt(1e-3))
    return f

def Benoit_System_2(u):
    f = u[0] ** 2 + u[1] ** 2 + (1 - u[0] * u[1])**2 + np.random.normal(0., np.sqrt(1e-3))
    return f


def con1_system(u):
    g1 = 1. - u[0] + u[1] ** 2 + 2. * u[1] - 2. + np.random.normal(0., np.sqrt(1e-3))
    return -g1


def con1_system_tight(u):
    g1 = 1. - u[0] + u[1] ** 2 + 2. * u[1] + np.random.normal(0., np.sqrt(1e-3))
    return -g1


def Benoit_System_noiseless_1(u):
    f = u[0] ** 2 + u[1] ** 2 + u[0] * u[1]  # + np.random.normal(0., np.sqrt(1e-3))
    return f

def Benoit_System_noiseless_2(u):
    f = u[0] ** 2 + u[1] ** 2 + (1 - u[0] * u[1])**2  # + np.random.normal(0., np.sqrt(1e-3))
    return f


def con1_system_noiseless(u):
    g1 = 1. - u[0] + u[1] ** 2 + 2. * u[1] - 2.  # + np.random.normal(0., np.sqrt(1e-3))
    return -g1


def con1_system_tight_noiseless(u):
    g1 = 1. - u[0] + u[1] ** 2 + 2. * u[1]  # + np.random.normal(0., np.sqrt(1e-3))
    return -g1


## B. Optimization on cost function

Optimization algorithm on cost function to find optimized input u given parameter

In [85]:
# Optimization on cost function
def cost_optimize(theta,u0):
    con = ({'type': 'ineq', 
            'fun': lambda u: con1_model(u,theta)}) 
    result = minimize(Benoit_Model,
                    u0,
                    constraints= con,
                    method='SLSQP',
                    options={'ftol': 1e-9},
                    args= (theta))

    return result.x,result.fun

In [86]:
# Test
u0 = [1,1] # Initial guess for optimization algorithm
theta = [0.5,0.5,0.5,0.5]
u,fun = cost_optimize(theta,u0)
print(f"optimal input: {u}, optimal output: {fun}")


optimal input: [ 2.00000000e+00 -1.89234472e-08], optimal output: 1.9999999999530884


## C. Model Adaptation

We try to minimize difference between output of an actual plant and output of a model. 

The difference is measured by SSE (Sum of Squared Error), which is used for cost function to be minimized

In [87]:
# cost function of adaptation
def SSE(theta,u,plant_func,plant_constraint):

    func_diff = plant_func(u) - Benoit_Model(u,theta)
    con_diff = plant_constraint(u) - con1_model(u,theta)

    return func_diff**2+con_diff**2

# Model Adaptation (Optimization of Parameters)
def SSE_optimize(plant_func,plant_constraint,theta0,u):

    result = minimize(SSE,
                      theta0,
                      method='SLSQP',
                      args= (u,plant_func,plant_constraint),
                      options= {'ftol': 1e-9})
    
    return result.x,result.fun


In [88]:
# test for adaptation cost function
x = SSE(theta = [0.5,0.5,0.5,0.5],
        u = [2,0],
        plant_func = Benoit_System_noiseless_1,
        plant_constraint =con1_system_noiseless)

print(f"cost function:{x}")
print("------------")

# test for model adaptation method
theta,fun = SSE_optimize(plant_func=Benoit_System_noiseless_1,
                     plant_constraint=con1_system_noiseless,
                     theta0=[0.5,0.5,0.5,0.5],
                     u=[2.00000000e+00,-1.89234472e-08])

print(f"optimal hyperparameters:{theta},\noptimal function: {fun}")



cost function:13.0
------------
optimal hyperparameters:[0.99999998 0.5        2.00000001 0.5       ],
optimal function: 1.1170386694296845e-15


## D. Overall Algorithm

### I. Model Adaptation on "Benoit_System_noiseless_1" plant system with "con1_system_noiseless" constraint

In [89]:
# Initial Guess (k=0)
uk = [1,1]
thetak = [0.5,0.5,0.5,0.5]

# dictionary: uk_1, fun, thetak_1, cost, 

for i in range(5):


    # Model Optimization
    uk_1,fun = cost_optimize(theta=thetak,u0=uk)
    print(f"optimal input: {uk_1}, optimal output: {fun}")
    
    # Model Adaptation
    thetak_1,cost = SSE_optimize(plant_func=Benoit_System_noiseless_1,
                        plant_constraint=con1_system_noiseless,
                        theta0=thetak,
                        u=uk_1)
    print(f"optimal hyperparameters:{thetak_1}, cost: {cost}")

    uk = uk_1
    thetak = thetak_1



optimal input: [ 2.00000000e+00 -1.89234472e-08], optimal output: 1.9999999999530884
optimal hyperparameters:[0.99999998 0.5        2.00000001 0.5       ], cost: 1.1201679861431565e-15
optimal input: [ 4.99999997e-01 -2.01114322e-09], optimal output: 0.2499999930048407
optimal hyperparameters:[0.99999998 0.5        5.00000001 0.5       ], cost: 1.021228093130283e-16
optimal input: [ 2.00000000e-01 -7.58719882e-09], optimal output: 0.039999999147657936
optimal hyperparameters:[ 0.99999998  0.5        10.99999956  0.5       ], cost: 1.1769003276782473e-14
optimal input: [ 9.09090946e-02 -7.58719882e-09], optimal output: 0.008264463333250038
optimal hyperparameters:[ 0.99999998  0.5        23.00000238  0.5       ], cost: 7.926616316173199e-14
optimal input: [ 4.34782564e-02 -7.35436818e-09], optimal output: 0.0018903587452594004
optimal hyperparameters:[ 0.99999998  0.5        47.00000596  0.5       ], cost: 1.4051593931378817e-15


### Actual Optimized Input and Output

In [90]:
u0 = [1,1] 
con = ({'type': 'ineq', 
        'fun': lambda u: con1_system_noiseless(u)}) 
result = minimize(Benoit_System_noiseless_1,
                u0,
                constraints= con,
                method='SLSQP',
                options={'ftol': 1e-9})

u = result.x
fun = result.fun

print(f"optimal input: {u}, optimal output: {fun}")

optimal input: [0. 0.], optimal output: 0.0


### Result Analysis

Notice that actual optimal input is [0,0] while the optimal input from model adaption converges to the model. However, the convergence is done with wrong hyperparameters especially on constraint con1_system_noiseless; notice constant increase in theta[2] (=47 at the last iteration). 

Moreover, the hyperparameters theta[1], theta[3] (=0.5,0.5) does not change because intermidiate optimal inputs have u[1] = 0 always. If the initial values were [0.1,0.1,0.1,0.1] then theta[1], theta[3] = 0.1 always. This makes hyperparameter to not get changed in model adaptation step.

### II. Model Adaptation on "Benoit_System_noiseless_1" plant system with "con1_system_tight_noiseless" constraint

In [91]:
# Initial Guess (k=0)
uk = [1,1]
thetak = [0.5,0.5,0.5,0.5]

# dictionary: uk_1, fun, thetak_1, cost, 

for i in range(5):

    # Model Optimization
    uk_1,fun = cost_optimize(theta=thetak,u0=uk)
    print(f"optimal input: {uk_1}, optimal output: {fun}")
    
    # Model Adaptation
    thetak_1,cost = SSE_optimize(plant_func=Benoit_System_noiseless_1,
                        plant_constraint=con1_system_tight_noiseless,
                        theta0=thetak,
                        u=uk_1)
    print(f"optimal hyperparameters:{thetak_1}, cost: {cost}")

    uk = uk_1
    thetak = thetak_1


optimal input: [ 2.00000000e+00 -1.89234472e-08], optimal output: 1.9999999999530884
optimal hyperparameters:[0.99999999 0.5        1.00000001 0.5       ], cost: 2.6598920050906437e-16
optimal input: [ 9.99999986e-01 -7.04835149e-08], optimal output: 0.9999999604125535
optimal hyperparameters:[0.99999999 0.5        1.00000001 0.5       ], cost: 1.959903026977365e-14
optimal input: [ 9.99999986e-01 -7.04835149e-08], optimal output: 0.9999999604125535
optimal hyperparameters:[0.99999999 0.5        1.00000001 0.5       ], cost: 1.959903026977365e-14
optimal input: [ 9.99999986e-01 -7.04835149e-08], optimal output: 0.9999999604125535
optimal hyperparameters:[0.99999999 0.5        1.00000001 0.5       ], cost: 1.959903026977365e-14
optimal input: [ 9.99999986e-01 -7.04835149e-08], optimal output: 0.9999999604125535
optimal hyperparameters:[0.99999999 0.5        1.00000001 0.5       ], cost: 1.959903026977365e-14


### Actual Optimized Input and Output

In [92]:
u0 = [1,1] 
con = ({'type': 'ineq', 
        'fun': lambda u: con1_system_tight_noiseless(u)}) 
result = minimize(Benoit_System_noiseless_1,
                u0,
                constraints= con,
                method='SLSQP',
                options={'ftol': 1e-9})

u = result.x
fun = result.fun

print(f"optimal input: {u}, optimal output: {fun}")

optimal input: [ 0.3684571  -0.39299332], optimal output: 0.14540320807022292


### Result Analysis

TO BE WRITTEN:

The result seems to show the lack of model flexibility, which prevents the optimal input from model to conver to actual optimal input. 

# 2. Modifier Adaptation

## A. Benoit Problem

In [93]:
# Plant Model 
def Benoit_Model(u,theta,modifier):

    f = theta[0] * u[0] ** 2 + theta[1] * u[1] ** 2 + np.sum(modifier[2] * u)
    return f


def con1_model(u,theta,modifier):

    g1 = 1. - theta[2]*u[0] + theta[3]*u[1] ** 2 + modifier[0][0] + modifier[0][1] + np.sum(modifier[1]*u)
    return -g1

# Actual Plant System
def Benoit_System_1(u):
    f = u[0] ** 2 + u[1] ** 2 + u[0] * u[1] + np.random.normal(0., np.sqrt(1e-3))
    return f

def Benoit_System_2(u):
    f = u[0] ** 2 + u[1] ** 2 + (1 - u[0] * u[1])**2 + np.random.normal(0., np.sqrt(1e-3))
    return f


def con1_system(u):
    g1 = 1. - u[0] + u[1] ** 2 + 2. * u[1] - 2. + np.random.normal(0., np.sqrt(1e-3))
    return -g1


def con1_system_tight(u):
    g1 = 1. - u[0] + u[1] ** 2 + 2. * u[1] + np.random.normal(0., np.sqrt(1e-3))
    return -g1


def Benoit_System_noiseless_1(u):
    f = u[0] ** 2 + u[1] ** 2 + u[0] * u[1]  # + np.random.normal(0., np.sqrt(1e-3))
    return f

def Benoit_System_noiseless_2(u):
    f = u[0] ** 2 + u[1] ** 2 + (1 - u[0] * u[1])**2  # + np.random.normal(0., np.sqrt(1e-3))
    return f


def con1_system_noiseless(u):
    g1 = 1. - u[0] + u[1] ** 2 + 2. * u[1] - 2.  # + np.random.normal(0., np.sqrt(1e-3))
    return -g1


def con1_system_tight_noiseless(u):
    g1 = 1. - u[0] + u[1] ** 2 + 2. * u[1]  # + np.random.normal(0., np.sqrt(1e-3))
    return -g1


## B. Optimization on Cost Function

In [95]:
# TO BE MODIFIED
def cost_optimize(u0,theta,modifier):

    con = ({'type': 'ineq',
            'fun': lambda u: con1_model(u,theta,modifier)})
    
    result = minimize((Benoit_Model),
                    u0,
                    constraints= con,
                    method='SLSQP',
                    options= {'ftol': 1e-9, 'disp': True},
                    args= (theta,modifier))
    
    return result.x,result.fun

In [99]:
# Test
u0 = np.array([1,1]) # Initial guess for optimization algorithm
modifier = np.array([[1,1],[1,1],[1,1]],dtype=object) # dtype=object: https://stackoverflow.com/questions/29877508/what-does-dtype-object-mean-while-creating-a-numpy-array
theta = np.array([0.5,0.5,0.5,0.5])

u,fun = cost_optimize(u0,theta,modifier)

print(f"optimal input: {u}, optimal output: {fun}")

Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.0000000000000355
            Iterations: 3
            Function evaluations: 9
            Gradient evaluations: 3
optimal input: [-5.         -1.00000001], optimal output: 7.0000000000000355


## C. Modifier Adaptation

In [101]:
# gradient modifier
def gradient_estimation(u,du,fun):
    
    # Predicted gradient from real plant
    gradient_u0 = (fun(u+[du,0]) - fun(u))/du
    gradient_u1 = (fun(u+[0,du]) - fun(u))/du
    gradient = [gradient_u0,gradient_u1]

    return gradient

# Modifier Update
def modifier_update(u,theta,modifier,du,plant_fun,plant_con1):
    
    # Calculate relavent bias and gradients:
    gradient_cost_p = gradient_estimation(u,du,plant_fun)
    ## Gradient of plant constraint function
    gradient_con1_p = gradient_estimation(u,du,plant_con1)
    ## Gradient of model cost function
    gradient_cost_m = approx_fprime(u,Benoit_Model,epsilon=1e-6,args=(theta,modifier))
    ## Gradient of model constraint function
    gradient_con1_m = approx_fprime(u,con1_model,epsilon=1e-6,args=(theta,modifier))
    ## Gradient of plant cost function


    epsil = plant_fun(u) - Benoit_Model(u,theta,modifier)
    lamda_0 = gradient_cost_p - gradient_cost_m 
    lamda_1 = gradient_con1_p - gradient_con1_m

    return [epsil,lamda_0,lamda_1]

# Modifier Adaptation
def modifier_adaptation(u,theta,modifier,du,plant_fun,plant_con1,K):
    new_modifier = modifier_update(u,theta,modifier,du,plant_fun,plant_con1)
    K_dimension = np.identity(np.shape(K)[0])
    modifier = (np.identity(K_dimension) - K)*modifier + K*new_modifier

    return modifier

In [ ]:
# Test
